# Part2: Preliminary Data Cleaning
<span style="color:red;">**Note: Since the work is divided into scraping and subsequent manual processing, the project code is split into five files. This is the second code file.**</span>
 
## 1. Merge Excel files collected from distributed scraping

In [1]:
import pandas as pd
# 列出所有文件名，这部分数据在文件夹“中间数据”中
file_names = [
    'Extracted_Data_LKH.xlsx',
    'Extracted_Data_PXL.xlsx',
    'Extracted_Data_ZS.xlsx',
    'Extracted_Data_LRZ.xlsx',
]

# 读取并合并所有文件
def merge_excel_files(file_list):
    data_frames = []
    for file_name in file_list:
        data_path = f'{file_name}'
        df = pd.read_excel(data_path)
        data_frames.append(df)
    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

# 合并数据并保存到一个新文件
combined_data = merge_excel_files(file_names)
combined_data.to_excel('数据清理1.0.xlsx', index=False)


## 2. Preparation



In [2]:
# 读取原始数据
data_path = '数据清理1.0.xlsx'
df = pd.read_excel(data_path)
df['original_index'] = df.index
# 记录原始数据量
original_count = df.shape[0]

df.head()

,股票代码,标题,发布时间,摘要内容,original_index
0,603567,珍宝岛：黑龙江珍宝岛药业股份有限公司2021年年度报告,2022-04-30,通过建设仓储物流体系、质检追溯体系、供应链金融体系、中药材标准化体系，打造优质优价的一手药材货源。,0
1,601825,沪农商行：上海农商银行2021年社会责任报告,2022-04-30,[一 经济责任]\n云南蒙自沪农商村镇银行多方联动构建供应链金融服务模式 | 为助力石榴产业...,1
2,601825,沪农商行：上海农村商业银行股份有限公司2021年度独立董事述职报告,2022-04-30,通过积极参加本行活动， 如参与战略规划及综合研究类课题评审， 指导本行开展投行联动、 供应链...,2
3,601825,沪农商行：上海农商银行2021年度报告,2022-04-30,本公司持续优化贷款结构，紧跟国家战略，围绕制造业、“3+6” 14 重点产业 、供应链金融、...,3
4,601825,沪农商行：上海农商银行2021年度报告摘要,2022-04-30,本公司持续优化贷款结构，紧跟国家战略，围绕制造业、“3+6”重点产业2、供应链金融、贸易融资...,4


## 3. Data Processing Code
### 3.1 Remove missing values and duplicates


In [3]:
# 删除“摘要内容”列中的缺失值
df = df.dropna(subset=['摘要内容'])
missing_removed_count = original_count - df.shape[0]  

# 删除重复数据：保留“摘要内容”中“|”出现次数最多的记录
df['pipe_count'] = df['摘要内容'].astype(str).apply(lambda x: x.count('|'))  
df = df.sort_values(by='pipe_count', ascending=False).drop_duplicates(subset=['股票代码', '发布时间'], keep='first')
duplicates_removed_count = df.shape[0]  
df.drop(columns='pipe_count', inplace=True)  

# 为“标题”和“股票代码”之间插入“公司名称”列
df.insert(1, "公司名称", df["标题"].apply(lambda x: x.split("：")[0] if "：" in x else x[:4]))

# 清理股票代码，只保留长度为6的代码，否则清空
df['股票代码'] = df['股票代码'].apply(lambda x: x if len(str(x)) == 6 else '')

# 计算“摘要内容”中“|”出现的次数并加1，添加为新列“关键词统计”
df['关键词统计'] = df['摘要内容'].astype(str).apply(lambda x: x.count('|') + 1)

# 分割“摘要内容”字段，创建多个内容列
max_splits = df['摘要内容'].astype(str).str.count(' \| ').max() + 1  
for i in range(1, max_splits + 1):
    df[f'内容{i}'] = df['摘要内容'].str.split(' \| ').str[i - 1]

# 计算重复删除后的净删除量
duplicates_removed_count = original_count - duplicates_removed_count

# 打印处理报告
print(f"删除了 {missing_removed_count} 个缺失值")
print(f"删除了 {duplicates_removed_count} 个重复值")
print(f"剩余 {df.shape[0]} 条数据")

# 保存处理后的数据
df.to_excel('数据清理1.1.xlsx', index=False)

删除了 540 个缺失值
删除了 5011 个重复值
剩余 6015 条数据



### 3.2 Further cleaning: remove additional duplicates


In [4]:
import pandas as pd
from difflib import SequenceMatcher

# 读取数据
data_path = '数据清理1.1.xlsx'
data = pd.read_excel(data_path)

# 相似度计算函数
def is_similar(a, b, threshold=0.9):
    return SequenceMatcher(None, a, b).ratio() >= threshold

# 检查并处理重复内容
for index, row in data.iterrows():
    # 保留第一次出现不相似的内容
    contents = [row[f'内容{i}'] for i in range(1, 6) if pd.notna(row[f'内容{i}'])]
    unique_contents = []
    to_remove = 0

    for content in contents:
        if all(not is_similar(content, unique_content) for unique_content in unique_contents):
            unique_contents.append(content)
        else:
            to_remove += 1  

    # 更新行内容
    for i in range(1, 6):
        if i <= len(unique_contents):
            data.at[index, f'内容{i}'] = unique_contents[i-1]
        else:
            data.at[index, f'内容{i}'] = pd.NA  


# 重新计算“关键词统计”，作为非空内容列的数量
content_columns = [col for col in df.columns if col.startswith('内容')]
df['关键词统计'] = df[content_columns].apply(lambda row: row.count(), axis=1)

# 保存处理后的数据到新的Excel文件
processed_data_path = '数据清理2.0.xlsx'
data.to_excel(processed_data_path, index=False)

processed_data_path

'数据清理2.0.xlsx'


## 4. Delete intermediate files

In [5]:
import os
files_to_delete = ['数据清理1.0.xlsx', '数据清理1.1.xlsx']
for file_name in files_to_delete:
    try:
        os.remove(file_name)
        print(f"文件 {file_name} 已删除。")
    except FileNotFoundError:
        print(f"找不到文件 {file_name}，可能已被删除。")
    except Exception as e:
        print(f"删除文件 {file_name} 时出错：{e}")

文件 数据清理1.0.xlsx 已删除。
文件 数据清理1.1.xlsx 已删除。
